In [1]:
import pymysql
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="1250",
    database="olist"
)

In [3]:
delivery_review_sql = """
    WITH reviews AS (
    SELECT
    	category_name_translation.product_category_name_english AS category_name,
    	AVG(reviews.review_score) AS avg_review_score
    FROM
        reviews
    INNER JOIN
    	order_items
    ON
        reviews.order_id = order_items.order_id
    INNER JOIN
    	products
    ON
    	order_items.product_id = products.product_id
    INNER JOIN
    	category_name_translation
    ON
    	products.product_category_name = category_name_translation.product_category_name
    GROUP BY
    	category_name
    ),
    delivery AS (
    SELECT
    	category_name_translation.product_category_name_english AS category_name,
        AVG(DATEDIFF(orders.order_delivered_customer_date, orders.order_purchase_timestamp)) AS actual_delivery_date,
        AVG(DATEDIFF(orders.order_delivered_customer_date, orders.order_estimated_delivery_date)) AS delay_delivery_date
    FROM
        orders
    INNER JOIN
    	order_items
    ON
        orders.order_id = order_items.order_id
    INNER JOIN
    	products
    ON
    	order_items.product_id = products.product_id
    INNER JOIN
    	category_name_translation
    ON
    	products.product_category_name = category_name_translation.product_category_name
    GROUP BY
    	category_name
    )
    SELECT
        reviews.category_name,
        reviews.avg_review_score,
        delivery.actual_delivery_date,
        delivery.delay_delivery_date
    FROM
        reviews
    INNER JOIN
        delivery
    ON
        reviews.category_name = delivery.category_name
"""

In [4]:
delivery_review_df = pd.read_sql(delivery_review_sql, conn)
delivery_review_df = delivery_review_df.set_index("category_name")
delivery_review_df.to_csv('delivery_review.csv', header=True)

C:\Users\minni\AppData\Local\Temp\ipykernel_32132\426138013.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  delivery_review_df = pd.read_sql(delivery_review_sql, conn)


In [5]:
delivery_review_df

avg_review_score  actual_delivery_date  \
category_name                                                        
agro_industry_and_commerce            4.0000               11.5825   
air_conditioning                      3.9692               12.1938   
art                                   3.9372               11.3046   
arts_and_craftmanship                 4.1250                5.6667   
audio                                 3.8255               13.2680   
...                                      ...                   ...   
stationery                            4.1939               12.6715   
tablets_printing_image                4.1235               12.9639   
telephony                             3.9469               12.7966   
toys                                  4.1586               11.6330   
watches_gifts                         4.0192               12.5885   

                            delay_delivery_date  
category_name                                    
agro_industry_and_commerce             -11.4612  
air_conditioning                       -14.1696  
art                                    -12.7766  
arts_and_craftmanship                   -6.7917  
audio                                  -10.1492  
...                                         ...  
stationery                             -12.0389  
tablets_printing_image                 -12.7711  
telephony                              -11.3826  
toys                                   -12.1032  
watches_gifts                          -11.9204  

[71 rows x 3 columns]

In [6]:
plt.style.use("seaborn-v0_8-pastel")
figure = plt.figure(figsize=(12, 6))
ax1 = figure.add_subplot(1, 2, 1)
ax2 = figure.add_subplot(1, 2, 2)

ax1.scatter(delivery_review_df["avg_review_score"], delivery_review_df["actual_delivery_date"])
ax2.scatter(delivery_review_df["avg_review_score"], delivery_review_df["delay_delivery_date"])

ax1.set_title("Actual Delivery Duration")
ax1.set_xlabel("Average Review Score")
ax1.set_ylabel("Delivery Duration")

ax2.set_title("Delay Delivery Duration")
ax2.set_xlabel("Average Review Score")
ax2.set_ylabel("Delivery Duration")

figure.suptitle("Average Review Score vs Delivery Duration", size=15)
plt.savefig("delivery_review.png", bbox_inches='tight')